# Determinism while using seeded randomness

As a part of comparing the different learners and algorithms, we have to consider the randomness in the environment. While working with randomness, it's important to consider that unlikely things do still happen! Hence even though there might be determinism in the form of a random number generator always generating the same values across different iterations, that doesn't mean it is necessarily mean it's deterministic across it's parameters. Below follows two examples, where one is deterministic across its arguments while the latter doesn't seem to be.

NB! It's important to notice how the random number generator is seeded/reset in the following examples, as we are running the exact same calls to randomness with the same random state, however the arguments to the randomness might be different.

In [1]:
import numpy as np

n_experiments = 10000
n_seeds = 10
n_total = n_experiments * n_seeds

click_ratios_1 = np.array([0.3, 0.7])
click_ratios_2 = np.array([0.7, 0.3])

## Deterministic across arguments

In the example below, we can see that the randomness will always affect the generation in a deterministic way, in the sense that if the ratios are different, they will all be influence "equally" by the randomness. I.e. if a ratio gets bigger, it will get bigger in the end aswell. By running this experiment, we can see that it will never fail.

In [2]:
def gen_fractions(rng, ratios):
    weight = rng.uniform(low=0.0, high=10.0)
    
    x = np.exp(ratios + weight)
    return x / np.sum(x)

n_failed_determ = 0

for seed in range(n_seeds):
    rng = np.random.default_rng(seed)
    for i in range(n_experiments):
        rng_state = rng.bit_generator.state
        fractions_1 = gen_fractions(rng, click_ratios_1)

        rng.bit_generator.state = rng_state
        fractions_2 = gen_fractions(rng, click_ratios_2)
        
        if fractions_2[0] < fractions_1[0]:
            # Even though the click ratio for the first 
            # element increased, the fraction still got smaller.
            n_failed_determ = n_failed_determ + 1

print(f"Fraction got smaller in {100 * (float(n_failed_determ) / n_total):.2f}% of the experiments!!")

Fraction got smaller in 0.00% of the experiments!!


## Non-deterministic across arguments

In this example however, we end up seeing failures due to the fact that the function `dirichlet` from numpy is not deterministic across its parameters. Hence, despite the click ratio increasing, that doesn't necessarily mean that the sampled fraction is greater (even for the exact same random state/seed). It's also worth noting, that even though it might rarely happen in some cases, it would still happen!

In [3]:
n_failed_dirich = 0

for seed in range(n_seeds):
    rng = np.random.default_rng(seed)
    for i in range(n_experiments):
        rng_state = rng.bit_generator.state
        fractions_1 = rng.dirichlet(click_ratios_1)

        rng.bit_generator.state = rng_state
        fractions_2 = rng.dirichlet(click_ratios_2)

        if fractions_2[0] < fractions_1[0]:
            # Even though the click ratio for the first 
            # element increased, the fraction still got smaller.
            # If dirichlet was deterministic across it's 
            # arguments, this wouldn't happen at all!!
            n_failed_dirich = n_failed_dirich + 1

print(f"Fraction got smaller in {100 * (float(n_failed_dirich) / n_total):.2f}% of the experiments!!")

Fraction got smaller in 2.27% of the experiments!!
